In [12]:
!pwd

/home/user-sl/Documents/metaptyxiako/Machine_Learning/Exc3


In [5]:
!pip3 install python-mnist

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

from mnist import MNIST

# In order to work unzip the MNIST files and put them under the same directory, i.e. /MNIST_dataset
Location = '/home/user-sl/Documents/metaptyxiako/Machine_Learning/Exc3/MNIST_dataset'

mndata = MNIST(Location)

images_train, labels_train = mndata.load_training()
# or
images_test, labels_test = mndata.load_testing()

print(np.array(images_train).shape)


(60000, 784)


In [2]:
from sklearn.preprocessing import MaxAbsScaler
#using the MaxAbsScaler to normalize our data to [0,1]
max_abs_scaler = MaxAbsScaler()
X_train_maxabs = max_abs_scaler.fit_transform(images_train)
X_test_maxabs = max_abs_scaler.transform(images_test)

print(labels_train[0])
print(X_train_maxabs[0])


5
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.       

In [3]:
from sklearn.preprocessing import MinMaxScaler
#using MinMaxScaler to normalize our data to [-1,1]
min_max_scaler = MinMaxScaler(feature_range=(-1,1))
X_train_minmax = min_max_scaler.fit_transform(images_train)
X_test_minmax = min_max_scaler.transform(images_test)

print(X_train_minmax[0])

[-1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         

In [46]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import time 

#train the [-1,1]
train_size=15000
#parameters to check
params_=[{"C": [1, 10, 100,1000],"kernel":['linear']},
         {"C": [1, 10, 100,1000],"gamma": [0.01, 0.001,'scale','auto'],"kernel":['rbf', 'sigmoid']},
         {"C": [1, 10, 100,1000],
         "degree":[1,5,10],
          "gamma": [0.01, 0.001,'scale','auto'],
          "kernel":['poly']}
        ]

t0 = time.time()
svc = GridSearchCV(SVC(),cv=5,param_grid=params_)  
svc.fit(X_train_minmax[:train_size,:], np.array(labels_train)[:train_size])  
print(svc.best_params_)  
svc_predict = time.time() - t0
#time that took to find the best parameter combination
print("SVc prediction for inputs in %.3f s"% (svc_predict))      

{'C': 100, 'degree': 10, 'gamma': 0.001, 'kernel': 'poly'}
SVc prediction for inputs in 15701.155 s


In [52]:
from sklearn.metrics import classification_report
#make predictions on the test set for [-1,1]
svc_predictions = svc.predict(X_train_minmax)
print(classification_report(np.array(labels_train), svc_predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5923
           1       0.99      0.99      0.99      6742
           2       0.97      0.98      0.98      5958
           3       0.98      0.97      0.98      6131
           4       0.98      0.98      0.98      5842
           5       0.98      0.97      0.97      5421
           6       0.99      0.99      0.99      5918
           7       0.99      0.98      0.98      6265
           8       0.96      0.98      0.97      5851
           9       0.98      0.97      0.97      5949

    accuracy                           0.98     60000
   macro avg       0.98      0.98      0.98     60000
weighted avg       0.98      0.98      0.98     60000



In [50]:
import csv

#code to pass all the results into a csv for [-1,1]
lk=svc.cv_results_.keys()
# print(lk)
with open('minmax_results.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(lk)
    row_counter=0
    total_rows=len(list(svc.cv_results_.values())[0])
    # print(len(list(svc.cv_results_.values())[0]))
    for i in range(total_rows):
      lv=[]
      for v in svc.cv_results_.values():
        lv.append(list(v)[i])
      writer.writerow(lv)


In [47]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import time 

#Check for the best parameter conbination and train the [0,1]
#the train size is 15k because it would take a lot of time to complete with more
train_size=15000
params_=[{"C": [1, 10, 100,1000],"kernel":['linear']},
         {"C": [1, 10, 100,1000],"gamma": [0.01, 0.001,'scale','auto'],"kernel":['rbf', 'sigmoid']},
         {"C": [1, 10, 100,1000],
         "degree":[1,5,10],
          "gamma": [0.01, 0.001,'scale','auto'],
          "kernel":['poly']}
        ]

t0 = time.time()
svc2 = GridSearchCV(SVC(),cv=5,param_grid=params_)  
svc2.fit(X_train_maxabs[:train_size,:], np.array(labels_train)[:train_size])  
print(svc2.best_params_)  
svc_predict2 = time.time() - t0
print("SVc prediction for inputs in %.3f s"% (svc_predict2))     

{'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}
SVc prediction for inputs in 27734.671 s


In [53]:
from sklearn.metrics import classification_report
#make predictions on the test set for [0,1]
svc_predictions2 = svc2.predict(X_train_maxabs)
print(classification_report(np.array(labels_train), svc_predictions2))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      5923
           1       0.99      0.99      0.99      6742
           2       0.97      0.98      0.97      5958
           3       0.97      0.97      0.97      6131
           4       0.97      0.98      0.98      5842
           5       0.97      0.97      0.97      5421
           6       0.99      0.99      0.99      5918
           7       0.98      0.98      0.98      6265
           8       0.97      0.97      0.97      5851
           9       0.98      0.96      0.97      5949

    accuracy                           0.98     60000
   macro avg       0.98      0.98      0.98     60000
weighted avg       0.98      0.98      0.98     60000



In [51]:
import csv

#code to pass all the results into a csv for [0,1]
lk=svc2.cv_results_.keys()
# print(lk)
with open('maxabs_results.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(lk)
    row_counter=0
    total_rows=len(list(svc2.cv_results_.values())[0])
    # print(len(list(svc2.cv_results_.values())[0]))
    for i in range(total_rows):
      lv=[]
      for v in svc2.cv_results_.values():
        lv.append(list(v)[i])
      writer.writerow(lv)


In [4]:
import numpy as np
from sklearn.svm import SVC
import time

t0 = time.time()
#train with full data and best parameter combination [-1,1]
#or replace the parameters with my first choice
# C= 100, degree= 10, gamma= 0.001, kernel='poly'
# C= 1, degree= 10, gamma= 'scale', kernel='poly'
svc_minmax_fd=SVC(C= 1, degree= 10, gamma= 'scale', kernel='poly') 
svc_minmax_fd.fit(X_train_minmax, np.array(labels_train)) 
train_minmax=time.time()-t0
print("Time for training: ",train_minmax)

Time for training:  316.7670681476593


In [5]:
from sklearn.metrics import classification_report
#make predictions on the train set for [-1,1]
svcfd_predictions_train= svc_minmax_fd.predict(X_train_minmax)
print(classification_report(np.array(labels_train), svcfd_predictions_train))
#make predictions on the test set for [-1,1]
svcfd_predictions_test= svc_minmax_fd.predict(X_test_minmax)
print(classification_report(np.array(labels_test), svcfd_predictions_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5923
           1       1.00      1.00      1.00      6742
           2       1.00      1.00      1.00      5958
           3       1.00      1.00      1.00      6131
           4       1.00      1.00      1.00      5842
           5       1.00      1.00      1.00      5421
           6       1.00      1.00      1.00      5918
           7       1.00      1.00      1.00      6265
           8       1.00      1.00      1.00      5851
           9       1.00      1.00      1.00      5949

    accuracy                           1.00     60000
   macro avg       1.00      1.00      1.00     60000
weighted avg       1.00      1.00      1.00     60000

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.99      0.99      1032
           3       0.99 

In [6]:
import numpy as np
from sklearn.svm import SVC
import time

t0 = time.time()
#train with full data and best parameter combination [0,1]
svc_maxabs_fd=SVC(C=100, gamma='scale', kernel='rbf')
svc_maxabs_fd.fit(X_train_maxabs, np.array(labels_train)) 
train_maxabs=time.time()-t0
print("Time for training: ",train_maxabs)

Time for training:  232.6112630367279


In [7]:
from sklearn.metrics import classification_report
#make predictions on the train set for [0,1]
svcfd_predictions_train2= svc_maxabs_fd.predict(X_train_maxabs)
print(classification_report(np.array(labels_train), svcfd_predictions_train2))
#make predictions on the test set for [0,1]
svcfd_predictions_test2= svc_maxabs_fd.predict(X_test_maxabs)
print(classification_report(np.array(labels_test), svcfd_predictions_test2))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5923
           1       1.00      1.00      1.00      6742
           2       1.00      1.00      1.00      5958
           3       1.00      1.00      1.00      6131
           4       1.00      1.00      1.00      5842
           5       1.00      1.00      1.00      5421
           6       1.00      1.00      1.00      5918
           7       1.00      1.00      1.00      6265
           8       1.00      1.00      1.00      5851
           9       1.00      1.00      1.00      5949

    accuracy                           1.00     60000
   macro avg       1.00      1.00      1.00     60000
weighted avg       1.00      1.00      1.00     60000

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.98 

In [8]:
from sklearn.metrics import accuracy_score

print("More precision accuracy (60k Accuracy)!!\n")
print("Accuracy for minmax-train_set: ",accuracy_score(np.array(labels_train),svcfd_predictions_train))
print("Accuracy for minmax-test_set: ",accuracy_score(np.array(labels_test),svcfd_predictions_test))

print("###########################################")

print("Accuracy for maxabs-train_set: ",accuracy_score(np.array(labels_train),svcfd_predictions_train2))
print("Accuracy for maxabs-test_set: ",accuracy_score(np.array(labels_test),svcfd_predictions_test2))


More precision accuracy (60k Accuracy)!!

Accuracy for minmax-train_set:  1.0
Accuracy for minmax-test_set:  0.9864
###########################################
Accuracy for maxabs-train_set:  1.0
Accuracy for maxabs-test_set:  0.9833


In [9]:
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import time
from sklearn.svm import SVC

# The percents of PCAs we want to check
percs=[0.95,0.80,0.60,0.50]

#Our best model for [0,1]
svc_maxabs_fd=SVC(C=100, gamma='scale', kernel='rbf')
#Our best model for [-1,1]
svc_minmax_fd=SVC(C= 1, degree= 10, gamma= 'scale', kernel='poly')

print(X_train_maxabs.shape)
print(X_test_maxabs.shape)
for p in percs:
    print("PCA-",p)
    pca=PCA(p)
    t0 = time.time()
    train_maxabs=pca.fit_transform(X_train_maxabs)
    test_maxabs=pca.transform(X_test_maxabs)
    print(train_maxabs.shape)
    print(test_maxabs.shape)
    svc_maxabs_fd.fit(train_maxabs, np.array(labels_train)) 
    print("Fit time for PCA",p,"(maxabs): ",time.time()-t0)
    
    svcfd_predictions_train2= svc_maxabs_fd.predict(train_maxabs)
    svcfd_predictions_test2= svc_maxabs_fd.predict(test_maxabs)
    print("Accuracy for maxabs-train_set: ",accuracy_score(np.array(labels_train),svcfd_predictions_train2))
    print("Accuracy for maxabs-test_set: ",accuracy_score(np.array(labels_test),svcfd_predictions_test2))
    print("_________________________________")
    
    pca2=PCA(p)
    t0 = time.time()
    train_minmax=pca2.fit_transform(X_train_minmax)
    test_minmax=pca2.transform(X_test_minmax)
    print(train_minmax.shape)
    print(test_minmax.shape)
    print("Done")
    svc_minmax_fd.fit(train_minmax, np.array(labels_train)) 
    print("Fit time for PCA",p,"(minmax): ",time.time()-t0)
    svcfd_predictions_train= svc_minmax_fd.predict(train_minmax)
    svcfd_predictions_test= svc_minmax_fd.predict(test_minmax)
    print("Accuracy for minmax-train_set: ",accuracy_score(np.array(labels_train),svcfd_predictions_train))
    print("Accuracy for minmax-test_set: ",accuracy_score(np.array(labels_test),svcfd_predictions_test))

(60000, 784)
(10000, 784)
PCA- 0.95
(60000, 154)
(10000, 154)
Fit time for PCA 0.95 (maxabs):  97.0656213760376
Accuracy for maxabs-train_set:  1.0
Accuracy for maxabs-test_set:  0.9854
_________________________________
(60000, 154)
(10000, 154)
Done
Fit time for PCA 0.95 (minmax):  884.3914382457733
Accuracy for minmax-train_set:  0.82815
Accuracy for minmax-test_set:  0.7133
PCA- 0.8
(60000, 44)
(10000, 44)
Fit time for PCA 0.8 (maxabs):  53.293978691101074
Accuracy for maxabs-train_set:  1.0
Accuracy for maxabs-test_set:  0.985
_________________________________
(60000, 44)
(10000, 44)
Done
Fit time for PCA 0.8 (minmax):  374.6982681751251
Accuracy for minmax-train_set:  0.8671833333333333
Accuracy for minmax-test_set:  0.8303
PCA- 0.6
(60000, 17)
(10000, 17)
Fit time for PCA 0.6 (maxabs):  26.16868257522583
Accuracy for maxabs-train_set:  0.9996333333333334
Accuracy for maxabs-test_set:  0.9725
_________________________________
(60000, 17)
(10000, 17)
Done
Fit time for PCA 0.6 (minm

In [ ]:
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import time
from sklearn.svm import SVC
# some code for testing the models and see their behaviour based on the PCAs

model=SVC(C= 100, degree= 10, gamma= 'scale', kernel='poly')
pca2=PCA(0.3)

t0 = time.time()
train_minmax=pca2.fit_transform(X_train_minmax)
test_minmax=pca2.transform(X_test_minmax)
print(train_minmax.shape)
print(test_minmax.shape)
print("Done")
model.fit(train_minmax, np.array(labels_train)) 
print("Fit time for PCA",0.3,"(minmax): ",time.time()-t0)
svcfd_predictions_train= model.predict(train_minmax)
svcfd_predictions_test= model.predict(test_minmax)
print("Accuracy for minmax-train_set: ",accuracy_score(np.array(labels_train),svcfd_predictions_train))
print("Accuracy for minmax-test_set: ",accuracy_score(np.array(labels_test),svcfd_predictions_test))

(60000, 5)
(10000, 5)
Done
